In [82]:
from enum import Enum
from collections import namedtuple
from functools import partial

In [83]:
def BFS(*, start, is_goal, get_neighbors):
    parent = {}
    to_visit = [start]
    discovered = set([start])

    while to_visit:
        vertex = to_visit.pop(0)

        if is_goal(vertex):
            path = []
            while vertex is not None:
                path.insert(0, vertex)
                vertex = parent.get(vertex)
                # print(path, vertex)
            
            return path
        
        for neighbor in get_neighbors(vertex):
            # print('\n')
            if neighbor not in discovered:
                discovered.add(neighbor)
                parent[neighbor] = vertex
                to_visit.append(neighbor)

In [84]:
State = namedtuple("State", ["man", "cabbage", "goat", "wolf"])
Location = Enum("Location", ["A", "B"])

In [85]:
start_state = State(
    man = Location.A,
    cabbage = Location.A,
    goat = Location.A,
    wolf = Location.A
)

goal_state = State(
    man = Location.B,
    cabbage = Location.B,
    goat = Location.B,
    wolf = Location.B
)

In [86]:
goal_state
goal_state.__eq__(goal_state) # check the equality

True

In [87]:
def is_valid(state):
    goat_eats_cabage = (
        state.goat == state.cabbage
        and state.man != state.goat
    )

    wolf_eats_goat = (
        state.wolf == state.goat
        and state.man != state.wolf
    )

    invalid = goat_eats_cabage or wolf_eats_goat
    
    return not invalid

In [94]:
def next_states(state):
    if state.man == Location.A:
        other_side = Location.B
    else:
        other_side = Location.A

    # print(state)
    # print(other_side)

    move = partial(state._replace, man = other_side)
    
    candidates = [move()]

    # print(move())

    for thing in ["cabbage", "goat", "wolf"]:
        # print(getattr(state, thing), state.man)
        if getattr(state, thing) == state.man:
            candidates.append(move(**{thing: other_side}))
            # print(other_side)
            # print(other_side)
            # print(move(**{thing: other_side}))
            # print(candidates)

    yield from filter(is_valid, candidates)

In [95]:
path = BFS(
    start = start_state,
    is_goal = goal_state.__eq__,
    get_neighbors = next_states
)

In [96]:
def describe_solutions(path):
    for old, new in zip(path, path[1:]):
        boat = [thing for thing in ["man", "cabbage", "goat", "wolf"]
                if getattr(old, thing) != getattr(new, thing)    
            ]
        
        print(old.man, "to", new.man, boat)

In [97]:
describe_solutions(path)

Location.A to Location.B ['man', 'goat']
Location.B to Location.A ['man']
Location.A to Location.B ['man', 'cabbage']
Location.B to Location.A ['man', 'goat']
Location.A to Location.B ['man', 'wolf']
Location.B to Location.A ['man']
Location.A to Location.B ['man', 'goat']
